## Generate Climos

This process runs [generate_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_climos-generate-climatological-means), which creates files with climatological means/standard deviations of input data from a netcdf file.

In [1]:
from birdy import WPSClient
from pkg_resources import resource_filename
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from bs4 import BeautifulSoup
import json

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

# Set up wps application
url = 'http://localhost:5001/wps'
thunderbird = WPSClient(url=url)

In [3]:
# Check info on `generate_climos` process
thunderbird.generate_climos?

Signature:
thunderbird.generate_climos(
    netcdf,
    operation,
    dry_run=None,
    convert_longitudes=True,
    split_vars=True,
    split_intervals=True,
    loglevel='INFO',
    climo=None,
    resolutions=None,
)
Docstring:
Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
operation : {'mean', 'std'}string
    Operation to perform on the datasets
climo : {'6190', '7100', '8110', '2020', '2050', '2080'}string
    Year ranges
resolutions : {'all', 'yearly', 'seasonal', 'monthly'}string
    Temporal Resolutions
convert_longitudes : boolean
    Transform longitude range from [0, 360) to [-180, 180)
split_vars : boolean
    Generate a separate file for each dependent variable in the file
split_intervals : boolean
    Gener

In [ ]:
# Dry run process
seasonal_opendap = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc'
annual_local = resource_filename('tests', 'data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc')
operation = 'mean'
climo = '6190'
resolutions = 'yearly'
dry_run = True

dry_output = thunderbird.generate_climos(
    netcdf=seasonal_opendap, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

In [ ]:
# Process dry run output
req = requests.get(dry_output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

In [ ]:
# generate climos
output = thunderbird.generate_climos(
    netcdf=seasonal_opendap, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

In [ ]:
# Process normal output
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl